In [1]:
import os
import subprocess

import numpy as np

from hyperopt import hp
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.layers import BatchNormalization, Flatten, Dense
from keras.layers import Input, Conv2D, Convolution2D
from keras.layers import Activation, concatenate, Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.applications.imagenet_utils import _obtain_input_shape

import Augmentor
import json

import tuner
from tuner import utils
from tuner import load_data
from tuner import augment_data
from tuner import use_hyperas
from tuner import net
from tuner.dataset import ClassificationDataset, AugmentDataset

from datetime import datetime

Using TensorFlow backend.
/home/fytroo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from easydict import EasyDict
params = EasyDict({
    'src_dir': '../micin-dataset/brain',
    'dst_dir': './brain',
    'bs': 32,
    'epochs': 10,
    'save_model_file': 'tuner.{}.model.hdf5'.format(int(datetime.now().timestamp()))
})
try:
    __file__
    import argparse
    argparse = argparse.ArgumentParser()
    parser.add_argument('-d','--dataset', dest='src_dir', type=str,
                         help='dataset directory')
    parser.add_argument('-o','--output', dest='save_model_file', type=str, default=params.save_model_file,
                         help='dataset directory')
    parser.add_argument('-b','--batchsize', dest='bs', type=int, default=params.bs,
                         help='batch size')
    parser.add_argument('-e','--epochs', type=int, default=params.epochs,
                         help='epochs')
    args = parser.parse_args()
except:
    args = params


### Convert  dataset from brain-dir to ready-dir 

In [3]:
df = load_data.df_fromdir_brain(args.src_dir)

In [4]:
load_data.classed_dir_fromdf(df, "tmp/brain")

### Load dataset via ready-dir

In [5]:
brain = ClassificationDataset('tmp/brain')

In [6]:
brain = AugmentDataset(brain)

In [7]:
brain.augmented_dir

'/home/fytroo/Tuner/dev/standard_datasets/5a67031cb037cb528757c444/auged'

In [8]:
brain.dataset.train_dir

'/home/fytroo/Tuner/dev/standard_datasets/5a67031cb037cb528757c444/train'

### Search best condition of data augmentation

In [9]:
brain.search_opt_augment(model=net.neoaug)

def data():
    train_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a67031cb037cb528757c444/train'
    test_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a67031cb037cb528757c444/validation'
    resize = '96'
    rescale = '1'
    df = load_data.df_fromdir_classed(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir_classed(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

         name label        handle  \
0    MS83.jpg    MS   MS/MS83.jpg   
1   MS068.jpg    MS  MS/MS068.jpg   
2   MS135.jpg    MS  MS/MS135.jpg   
3   MS167.jpg    MS  MS/MS167.jpg   
4   MS123.jpg    MS  MS/MS123.jpg   
5   PD050.jpg    PD  PD/PD050.jpg   
6   PD079.jpg    PD  PD/PD079.jpg   
7   PD102.jpg    PD  PD/PD102.jpg   
8   PD062.jpg    PD  PD/PD062.jpg   
9   PD077.jpg    PD 

Epoch 5/10
 - 0s - loss: 1.3839 - acc: 0.2545 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 6/10
 - 0s - loss: 1.3344 - acc: 0.3170 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 7/10
 - 0s - loss: 1.3676 - acc: 0.3705 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 8/10
 - 0s - loss: 1.3322 - acc: 0.3884 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 9/10
 - 0s - loss: 1.3666 - acc: 0.3080 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 10/10
 - 0s - loss: 1.3565 - acc: 0.3259 - val_loss: 12.0886 - val_acc: 0.2500
Test accuracy: 0.25
--------------------------------------------------------------------------------
Operations: 3
	0: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
	1: RandomErasing (probability=0.5 rectangle_area=0.2 )
	2: Shear (probability=0.3 max_shear_left=2 max_shear_right=2 )
Images: 0
Classes: 0

You can remove operations using the appropriate index and the remove_operation(index) function.
Epoch 1/10
 - 0s - loss: 1.3882 - acc: 0.3125 - val_loss: 11.2616 - val_acc: 0.25

Epoch 1/10
 - 1s - loss: 1.4264 - acc: 0.3705 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 2/10
 - 0s - loss: 1.3586 - acc: 0.3839 - val_loss: 11.2827 - val_acc: 0.3000
Epoch 3/10
 - 0s - loss: 1.3172 - acc: 0.4107 - val_loss: 12.0860 - val_acc: 0.2500
Epoch 4/10
 - 0s - loss: 1.2878 - acc: 0.4509 - val_loss: 11.9174 - val_acc: 0.2500
Epoch 5/10
 - 0s - loss: 1.2621 - acc: 0.4241 - val_loss: 10.7539 - val_acc: 0.3000
Epoch 6/10
 - 0s - loss: 1.2238 - acc: 0.4643 - val_loss: 11.8330 - val_acc: 0.2500
Epoch 7/10
 - 0s - loss: 1.1505 - acc: 0.5580 - val_loss: 11.2827 - val_acc: 0.3000
Epoch 8/10
 - 0s - loss: 1.1225 - acc: 0.5536 - val_loss: 10.4768 - val_acc: 0.3500
Epoch 9/10
 - 0s - loss: 1.0552 - acc: 0.6116 - val_loss: 9.5703 - val_acc: 0.4000
Epoch 10/10
 - 0s - loss: 0.9799 - acc: 0.6250 - val_loss: 9.6709 - val_acc: 0.4000
Test accuracy: 0.40000000596
--------------------------------------------------------------------------------
Operations: 2
	0: Flip (probability=0.5 top_bottom_

### Execute data augmentation with best condition

In [11]:
brain.augment_dataset()

Processing MS070.jpg:   5%|▌         | 10/184 [00:00<00:02, 59.00 Samples/s]

Initialised with 46 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a67031cb037cb528757c444/train/MS/output.

Processing PD044.jpg:   3%|▎         | 5/184 [00:00<00:05, 33.74 Samples/s]  

Initialised with 46 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a67031cb037cb528757c444/train/PD/output.

Processing N080.jpg:  10%|█         | 19/184 [00:00<00:01, 102.79 Samples/s] 

Initialised with 79 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a67031cb037cb528757c444/train/N/output.

Processing PS107.jpg:   3%|▎         | 5/184 [00:00<00:05, 33.85 Samples/s] 

Initialised with 75 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a67031cb037cb528757c444/train/PS/output.

### Tuning hyper parameter of CNN

In [14]:
best_condition, best_model = use_hyperas.exec_hyperas(
    brain.augmented_dir,
    brain.dataset.validation_dir,
    net.simplenet,
    batch_size = 32,
    epochs=10,
    optimizer='adam',
    rescale=1./255
)
best_model.save(args.save_model_file)

def data():
    train_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a67031cb037cb528757c444/auged'
    test_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a67031cb037cb528757c444/validation'
    resize = '96'
    rescale = '0.00392156862745098'
    df = load_data.df_fromdir_classed(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir_classed(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

         name label        handle  \
0    MS83.jpg    MS   MS/MS83.jpg   
1   MS068.jpg    MS  MS/MS068.jpg   
2   MS135.jpg    MS  MS/MS135.jpg   
3   MS167.jpg    MS  MS/MS167.jpg   
4   MS123.jpg    MS  MS/MS123.jpg   
5   PD050.jpg    PD  PD/PD050.jpg   
6   PD079.jpg    PD  PD/PD079.jpg   
7   PD102.jpg    PD  PD/PD102.jpg   
8   PD062.jpg    PD  PD/PD062.jpg   
9 

Test accuracy: 0.40000000596
Train on 736 samples, validate on 20 samples
Epoch 1/10
 - 2s - loss: 1.4356 - acc: 0.2649 - val_loss: 1.3765 - val_acc: 0.2500
Epoch 2/10
 - 1s - loss: 1.3690 - acc: 0.3071 - val_loss: 1.3037 - val_acc: 0.4000
Epoch 3/10
 - 1s - loss: 1.2907 - acc: 0.3967 - val_loss: 1.5654 - val_acc: 0.3500
Epoch 4/10
 - 1s - loss: 1.1773 - acc: 0.4932 - val_loss: 1.3749 - val_acc: 0.4000
Epoch 5/10
 - 1s - loss: 0.9796 - acc: 0.5856 - val_loss: 2.2032 - val_acc: 0.4500
Epoch 6/10
 - 1s - loss: 0.8010 - acc: 0.6848 - val_loss: 2.0357 - val_acc: 0.4000
Epoch 7/10
 - 1s - loss: 0.5693 - acc: 0.8071 - val_loss: 3.1917 - val_acc: 0.3500
Epoch 8/10
 - 1s - loss: 0.3462 - acc: 0.8723 - val_loss: 2.7005 - val_acc: 0.4000
Epoch 9/10
 - 1s - loss: 0.2152 - acc: 0.9321 - val_loss: 4.1056 - val_acc: 0.4000
Epoch 10/10
 - 1s - loss: 0.1615 - acc: 0.9511 - val_loss: 4.4698 - val_acc: 0.3500
Test accuracy: 0.34999999404
Train on 736 samples, validate on 20 samples
Epoch 1/10
 - 2s - lo